In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame
import csv
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import scipy.ndimage
from glob import glob
from natsort import natsorted
from numpy import savetxt
import os
sns.set()
%matplotlib inline
%precision 3

'%.3f'

# Calculation of cortical convergence along x- and y- axes

In [66]:
#exporting compx, compy as a matrix
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import scipy.ndimage
from glob import glob
from natsort import natsorted
from numpy import savetxt

def divergence1(field):
    return np.gradient(field, axis =1)/16 #16 is a spacing between grids in PIV lab (unit pixel). Calculating gradient of flow velocity along x-axis
def divergence2(field):
    return np.gradient(field, axis =0)/16 #Calculating gradient of flow velocity along y-axis
def divergence3(u, v):
    return (np.gradient(u,  axis=1)+ np.gradient(v, axis=0))/16 #Calculating divergence.

samplename = [1, 2, 3, 4, 8, 9, 10, 12, 14, 16, 18, 19, 21, 24, 25, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 40]

for k in range(1, 28):    #27 embryos used
    numu = glob("./PIVdata/R"+str(samplename[k-1])+"fu*"+".csv")     #Reading file names of u vector (x-axis/furrow-directed flow). Exported CSV files should be in this directory
    numu2 = natsorted(numu)                             #Natural sorting of the file name
    u = [pd.read_csv(f, header = None).values for f in numu2] #Importing data from CSV files into "u"
    u_ = np.asarray(u)  #Converting data as array


    numv = glob("./PIVdata/R"+str(samplename[k-1])+"fv*"+".csv") #Same thing for the perpencidular axis flow data.
    numv2 = natsorted(numv)
    v = [pd.read_csv(f, header = None).values for f in numv2]
    v_ = np.asarray(v)

    
    for i in range(0,len(numu2)): 

        convx = divergence1(u_[i])*(-1) #Calculating convergence along perpendicular axis/x-axis
        convy = divergence2(v_[i])*(-1) #Calculating convergence along ring axis/y-axis

        savetxt('./perpendicularF/convx' + str(k)+'-'+str(i)+'.csv', convx, delimiter = ',') #create folder named "perpendicularF"
        savetxt('./perpendicularF/convy' + str(k)+'-'+str(i)+'.csv', convy, delimiter = ',')

        


# Calculating mean convergence and flow data in a specified region in a matrix. Grid position is adjusted relative to the ring centre at the surface so that the furrow position in difffrent samples are at the same grid

In [67]:
# Check PIV lab window and enter value n when your furrow center is nth grid from the left along x-axis. Enter for each sample.
xcen = [11,10,10,10,10,10,9,13, 10, 8 , 10, 10, 11, 11, 10, 10, 11, 9, 11, 10,10,10, 9, 10, 10, 10, 10 ] 
# Check PIV lab window and enter value n when your furrow center is nth grid from the top along y-axis. Enter for each sample.
ycen = [8, 9, 10, 8, 9, 10, 7, 12, 9, 7, 9, 8,9, 7, 9, 8, 7,8, 10, 8, 8, 10, 8, 8, 7, 8, 8 ] 

samplename = [1, 2, 3, 4, 8, 9, 10, 12, 14, 16, 18, 19, 21, 24, 25, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 40]

for k in range(1, 28):  #27 embryos used
    convx_meanf = []   #Mean perpendicular axis convergence  at the furrow centre will be stored here 
    convx_meanfl = []  #Mean perpendicular axis convergence  at the left center will be stored here 
    convx_meanfr = []  #Mean perpendicular axis convergence  at the right center will be stored here 
    convxlist = pd.DataFrame() 
    
    convy_meanf = [] #Mean ring axis convergence  at the furrow centre will be stored here 
    convy_meanfl = [] #Mean ring axis convergence  at the left center will be stored here 
    convy_meanfr = [] #Mean ring axis convergence  at the right center will be stored here 
    convylist = pd.DataFrame()
    
    u_meanf = [] #Mean perpendicular axis flow velocity  at the furrow centre will be stored here 
    u_meanfl = [] #Mean perpendicular axis flow velocity  at the left center will be stored here 
    u_meanfr = [] #Mean perpendicular axis flow velocity  at the right center will be stored here 
    ulist = pd.DataFrame()
    
    v_meanf = [] #Mean ring axis flow velocity  at the furrow centre will be stored here 
    v_meanfl = [] #Mean perpendicular axis flow velocity  at the left center will be stored here 
    v_meanfr = [] #Mean perpendicular axis flow velocity  at the right center will be stored here 
    vlist = pd.DataFrame()
    
    numconvx = glob("./perpendicularF/convx%i-*.csv" %k) #Importing convx matrix data as convx_
    numconvx2 = natsorted(numconvx)
    convx = [pd.read_csv(f).values for f in numconvx2]
    convx_ = np.asarray(convx)
    
    numconvy = glob("./perpendicularF/convy%i-*.csv" %k) #Importing convy matrix data as convy_
    numconvy2 = natsorted(numconvy)
    convy = [pd.read_csv(f).values for f in numconvy2]
    convy_ = np.asarray(convy)
    
    numu = glob("./PIVdata/R"+str(samplename[k-1])+"fu*"+".csv")  #Importing u matrix data as u_
    numu2 = natsorted(numu)
    u = [pd.read_csv(f).values for f in numu2]
    u_ = np.asarray(u)
    
    numv = glob("./PIVdata/R"+str(samplename[k-1])+"fv*"+".csv") #Importing v matrix data as v_
    numv2 = natsorted(numv)
    v = [pd.read_csv(f).values for f in numv2]
    v_ = np.asarray(v)


    for i in range(0,len(numconvx)):
        
        convx_meanf.append(convx_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-3:xcen[k-1]+3].mean())   #Calculating mean convx value at the center
        convx_meanfl.append(convx_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-5:xcen[k-1]-3].mean())   #Calculating mean convx value at the left
        convx_meanfr.append(convx_[i][ycen[k-1]-2:ycen[k-1]+1, xcen[k-1]+1:xcen[k-1]+3].mean()) #Calculating mean convx value at the right
        
        convy_meanf.append(convy_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-3:xcen[k-1]+3].mean())    #Calculating mean convy value at the center
        convy_meanfl.append(convy_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-5:xcen[k-1]-3].mean())    #Calculating mean convy value at the left
        convy_meanfr.append(convy_[i][ycen[k-1]-2:ycen[k-1]+1, xcen[k-1]+1:xcen[k-1]+3].mean())  #Calculating mean convy value at the right
       
        u_meanf.append(u_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-3:xcen[k-1]+3].mean())           #Same for u
        u_meanfl.append(u_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-5:xcen[k-1]-3].mean())
        u_meanfr.append(u_[i][ycen[k-1]-2:ycen[k-1]+1, xcen[k-1]+1:xcen[k-1]+3].mean())
         
        v_meanf.append(v_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-3:xcen[k-1]+3].mean())            #Same for v
        v_meanfl.append(v_[i][ycen[k-1]-2:ycen[k-1]+1,xcen[k-1]-5:xcen[k-1]-3].mean())
        v_meanfr.append(v_[i][ycen[k-1]-2:ycen[k-1]+1, xcen[k-1]+1:xcen[k-1]+3].mean())
        
        #Compiling data frame of above data
        convxlist = pd.DataFrame({'convx_meanf': convx_meanf, 'convx_meanfl':convx_meanfl, 'convx_meanfr': convx_meanfr}) 
        convylist = pd.DataFrame({'convy_meanf': convy_meanf, 'convy_meanfl':convy_meanfl, 'convy_meanfr': convy_meanfr})
        ulist = pd.DataFrame({'u_meanf': u_meanf, 'u_meanfl':u_meanfl, 'u_meanfr': u_meanfr})
        vlist = pd.DataFrame({'v_meanf': v_meanf, 'v_meanfl':v_meanfl, 'v_meanfr': v_meanfr})
        
        #Compiling one giant data frame containing all data
        flowR1 = pd.DataFrame()
        flowR1 = pd.concat([flowR1, convxlist], axis = 1)
        flowR1 = pd.concat([flowR1, convylist], axis =1)
        flowR1 = pd.concat([flowR1, ulist], axis =1)
        flowR1 = pd.concat([flowR1, vlist], axis =1)
        
    #Exporting csv files. k is the sample ID. n = k csv files will be generated if successful. In this example 27 csv files.
    flowR1.to_csv('./flow%01d.csv' %k) 


# Generation of a summary dataframe

In [68]:
#Orgnizing columns in flowRx.csv to make a Flowsummary.csv
numR = glob("./flow*.csv")
numR2 = natsorted(numR)
convx_fc= pd.DataFrame()
convx_fl = pd.DataFrame()
convx_fr = pd.DataFrame()

convy_fc= pd.DataFrame()
convy_fl = pd.DataFrame()
convy_fr = pd.DataFrame()

u_fc= pd.DataFrame()
u_fl = pd.DataFrame()
u_fr = pd.DataFrame()

v_fc= pd.DataFrame()
v_fl = pd.DataFrame()
v_fr = pd.DataFrame()

for i in range (0,27):
    convx_fc1= pd.DataFrame()
    convx_fl1= pd.DataFrame()
    convx_fr1 = pd.DataFrame()

    convy_fc1= pd.DataFrame()
    convy_fl1 = pd.DataFrame()
    convy_fr1 = pd.DataFrame()
    
    u_fc1= pd.DataFrame()
    u_fl1 = pd.DataFrame()
    u_fr1 = pd.DataFrame()

    v_fc1= pd.DataFrame()
    v_fl1 = pd.DataFrame()
    v_fr1 = pd.DataFrame()


    master = pd.DataFrame()
    k = i+1
    master = pd.read_csv(numR2[i])
    convx_fc1 = master.iloc[:, 1:2]
    convx_fc1.columns= ['convxfc%01d' %k]
    convx_fc = pd.concat([convx_fc, convx_fc1], axis =1)
    
    convx_fl1 = master.iloc[:, 2:3]
    convx_fl1.columns= ['convxfl%01d' %k]
    convx_fl = pd.concat([convx_fl, convx_fl1], axis =1)
    
    convx_fr1 = master.iloc[:, 3:4]
    convx_fr1.columns= ['convxfr%01d' %k]
    convx_fr = pd.concat([convx_fr, convx_fr1], axis =1)
    
    convy_fc1 = master.iloc[:, 4:5]
    convy_fc1.columns= ['convyfc%01d' %k]
    convy_fc = pd.concat([convy_fc, convy_fc1], axis =1)
    
    convy_fl1 = master.iloc[:, 5:6]
    convy_fl1.columns= ['convyfl%01d' %k]
    convy_fl = pd.concat([convy_fl, convy_fl1], axis =1)
    
    convy_fr1 = master.iloc[:, 6:7]
    convy_fr1.columns= ['convyfr%01d' %k]
    convy_fr = pd.concat([convy_fr, convy_fr1], axis =1)
    
    u_fc1 = master.iloc[:, 7:8]
    u_fc1.columns= ['u%01d' %k]
    u_fc = pd.concat([u_fc, u_fc1], axis =1)
    
    u_fl1 = master.iloc[:, 8:9]
    u_fl1.columns= ['ufl%01d' %k]
    u_fl = pd.concat([u_fl, u_fl1], axis =1)
    
    u_fr1 = master.iloc[:, 9:10]
    u_fr1.columns= ['ufr%01d' %k]
    u_fr = pd.concat([u_fr, u_fr1], axis =1)
    
    v_fc1 = master.iloc[:, 10:11]
    v_fc1.columns= ['v%01d' %k]
    v_fc = pd.concat([v_fc, v_fc1], axis =1)
    
    v_fl1 = master.iloc[:, 11:12]
    v_fl1.columns= ['vfl%01d' %k]
    v_fl = pd.concat([v_fl, v_fl1], axis =1)
    
    v_fr1 = master.iloc[:, 12:13]
    v_fr1.columns= ['vfr%01d' %k]
    v_fr = pd.concat([v_fr, v_fr1], axis =1)
    


Flowdata = pd.concat([convx_fc, convx_fl, convx_fr, convy_fc, convy_fl, convy_fr, u_fc, u_fl, u_fr,v_fc, v_fl, v_fr  ], axis =1)
Flowdata

,convxfc1,convxfc2,convxfc3,convxfc4,convxfc5,convxfc6,convxfc7,convxfc8,convxfc9,convxfc10,...,vfr18,vfr19,vfr20,vfr21,vfr22,vfr23,vfr24,vfr25,vfr26,vfr27
0,0.007824,0.001083,0.008291,0.006717,0.006609,0.008698,0.005038,0.008093,0.000978,0.002874,...,-0.191118,0.115496,0.598377,0.204141,1.023398,0.293607,0.061291,-0.306145,0.764767,-0.056193
1,0.008317,-0.001162,0.002585,0.012914,0.012799,0.012339,0.004979,0.002505,-0.000702,0.003280,...,-0.337795,0.002892,0.059907,0.377777,0.164718,-0.322650,-0.015455,-0.119087,0.416716,-0.081677
2,0.018612,0.005004,0.006563,0.009958,0.013450,0.010664,0.010893,0.006821,0.006929,-0.002277,...,-0.280360,0.007040,-0.800395,-0.073944,0.261552,0.103574,0.005732,0.059986,-0.179883,-0.226249
3,0.008976,0.005377,0.000823,0.008924,0.007254,0.021728,0.002365,0.003972,0.004017,0.003031,...,-0.298896,-0.124536,-0.316105,-0.220576,0.198809,0.197739,0.064687,-0.145194,-0.449907,-0.035649
4,0.012915,0.002830,0.005288,0.004158,0.010749,0.003285,0.002961,-0.002995,0.005473,0.004065,...,-0.520082,-0.050185,-0.264334,-0.213901,0.378477,0.357143,-0.182885,0.018880,-0.090503,-0.202820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.008742,0.000185,0.001511,0.003053,-0.002143,-0.008842,0.006548,0.013841,0.007084,0.001290,...,0.999202,-0.112860,NaN,NaN,NaN,-0.099425,0.014814,-0.129121,NaN,-0.567677
115,0.007662,-0.000988,-0.008507,0.019050,0.002633,-0.004917,0.011295,0.010710,0.011073,0.003624,...,-0.355945,-0.271557,NaN,NaN,NaN,-0.290608,0.193631,-0.913120,NaN,-0.416532
116,0.015965,-0.010506,-0.006525,0.013756,0.017785,-0.002895,-0.001952,0.009213,0.005824,0.001793,...,-0.499308,-0.128645,NaN,NaN,NaN,-0.405330,-0.002932,0.164443,NaN,0.493257
117,0.012154,0.000645,-0.000643,0.003306,0.007637,-0.011499,-0.008110,0.008343,0.002876,0.003882,...,-0.302607,-0.135226,NaN,NaN,NaN,-0.172872,0.212741,0.178195,NaN,-0.334728


# Export Flowdata as a csv file.

In [69]:
Flowdata.to_csv('./Flowsummary-controlleadingedge.csv')

# Preparation for time alignment relative to 10% ring closure

In [3]:
numR = glob("./Ringtexts/R*new.txt")  #getting a list of "R*.txt" files.This is created by Step4-macro.
numR2 = natsorted(numR)              #natural sort order of files (normal sort: 0, 1, 10, 11, 2, 3, 4..., natural sort: 0, 1, 2, 3, ..., 11, 12..)
numR2
Ring = pd.DataFrame()
Shift = pd.DataFrame()
Leading = pd.DataFrame()
Lagging = pd.DataFrame()
Time = [0]
samplename=[1,2,3,4,8,9,10,12,14,16,18,19,21,24,25,27,28,29,31,32,33,34,36,37,38,39,40]
for i in range (0,27):
    Ring1 = pd.DataFrame()
    Shift1 = pd.DataFrame()
    Leading1 = pd.DataFrame()
    Lagging1 = pd.DataFrame()
    Time1 = [0]
    k = i+1
    Ring0 = pd.read_csv("./Ringtexts/R"+str(samplename[k-1])+"new.txt", delimiter = ' ')   #numR2 is a list of file name numR2[0] is R1.txt Delimiter is a space
    Ring1 = Ring0.iloc[:, 1:2]
    Ring1.columns= ["R"+str(samplename[k-1])]
    Ring = pd.concat([Ring, Ring1], axis =1)
    
    
    Shift1 = Ring0.iloc[:, 7:8]
    Shift1.columns= ['S'+str(samplename[k-1])]
    Shift = pd.concat([Shift, Shift1], axis =1)
    
    Leading1 = Ring0.iloc[:, 11:12]
    Leading1.columns= ['Lead'+str(samplename[k-1])]
    Leading = pd.concat([Leading, Leading1], axis =1)
    
    Lagging1 = Ring0.iloc[:, 12:13]
    Lagging1.columns= ['Lag'+str(samplename[k-1])]
    Lagging = pd.concat([Lagging, Lagging1], axis =1)
    
    Time0 = Ring0[Ring0.Relativeringdiameter > 10] #this value should be changed if your data should be aligned relative to different values.
    Time1 = Time0.iloc[0,0]
    Time = np.append(Time,Time1)


Ringdata = pd.concat([Ring, Shift], axis =1)
Ringdata = pd.concat([Ringdata, Leading], axis =1)
Ringdata = pd.concat([Ringdata, Lagging], axis =1)
Ringdata

,R1,R2,R3,R4,R8,R9,R10,R12,R14,R16,...,Lag29,Lag31,Lag32,Lag33,Lag34,Lag36,Lag37,Lag38,Lag39,Lag40
0,0.0000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.1637,0.15350,0.2270,0.14370,0.08748,-0.05845,-0.05045,-0.09524,-0.03759,0.2283,...,-0.1260,-0.04681,-0.002822,-0.03092,-0.04545,-0.1114,-0.05081,-0.05196,-0.015650,-0.004416
2,-0.2839,-0.05897,0.2140,0.18410,0.14450,-0.13650,0.09533,-0.69270,-0.09298,0.6893,...,-0.1571,-0.10200,-0.039480,0.05767,-0.06585,-0.3869,-0.08741,-0.27160,0.000323,-0.013240
3,-0.4868,-0.49200,-0.0417,0.14530,0.16520,-0.17710,0.16120,-1.55190,-0.15720,0.8276,...,-0.2458,-0.16710,-0.093730,0.12020,-0.22820,-0.6395,0.06215,-0.65930,-0.129900,-0.371700
4,-0.8443,-0.85820,-0.3960,0.03935,0.19520,-0.17080,-0.02569,-2.34130,-0.31830,0.8092,...,-0.2489,-0.10970,-0.214800,0.15510,-0.28580,-0.8438,-0.16830,-1.18560,-0.307300,-0.903800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Alignment of Flowdata timing relative to cytokinesis onset

In [4]:
Flowdata2 = pd.DataFrame()
Flowdata2 = pd.read_csv('./Flowsummary-controlleadingedge.csv')
timeshift = [0]


samplen = 27 #your sample number
parametern = 13 #this should not be changed
timeofRingat10 = Time[1:samplen+1]
Relativetime = [(x - max(timeofRingat10))*(-1) for x in timeofRingat10]
timeshift = [0]





for i in range(0, parametern):
    timeshift.extend(Relativetime)


Flowdata2.index = Flowdata2.index+abs(min(timeshift))
Flowdata2 = Flowdata2.reindex(np.arange(len(Flowdata2)))

for count, value in enumerate(Flowdata2.columns):
        Flowdata2[value]= Flowdata2[value].shift(timeshift[count])

Flowdata2.to_csv('./Flowsummary-controlleadingedgealigned.csv')

# If you need to calculate normalized time velocity use this

In [5]:
Flowdata3 = pd.read_csv('./Flowsummary-controlleadingedgealigned.csv')
Flowdata4 = Flowdata3*1.538638 #this value should be actual time interval divided by normalized time interval
Flowdata4.to_csv('./Flowsummary-controlleadingedgealignednormalized.csv')

# Peak ring-directed flow

In [12]:
data6 = pd.read_csv('./Flowsummary-controlleadingedgealigned.csv') #modify file name
targetcolumn = data6.columns.get_loc('ufl1') #ufl is the A-P axis flow at the area anterior to the furrow
peakeccentricity = []
for i in range(0,27):
    k = i
    peakeccentricity = np.append(peakeccentricity, data6.iloc[:,i+targetcolumn].max(axis=0))

peakeccentricitypd = pd.DataFrame(data= peakeccentricity)
peakeccentricitypd.to_csv('./Peakfurrowdirectedflowleft-leadingedge.csv')

# Peak A-P axis convergence

In [13]:
data6 = pd.read_csv('./Flowsummary-controlleadingedgealigned.csv') #modify file name
targetcolumn = data6.columns.get_loc('convxfc1')
peakeccentricity = []
for i in range(0,27):
    k = i
    peakeccentricity = np.append(peakeccentricity, data6.iloc[:,i+targetcolumn].max(axis=0))

peakeccentricitypd = pd.DataFrame(data= peakeccentricity)
peakeccentricitypd.to_csv('./PeakAPconv-leadingedge.csv')

# Peak circumferential-axis convergence

In [2]:
data6 = pd.read_csv('./Flowsummary-controlleadingedgealigned.csv') #modify file name
targetcolumn = data6.columns.get_loc('convyfc1')
peakeccentricity = []
for i in range(0,27):
    k = i
    peakeccentricity = np.append(peakeccentricity, data6.iloc[:,i+targetcolumn].max(axis=0))

peakeccentricitypd = pd.DataFrame(data= peakeccentricity)
peakeccentricitypd.to_csv('./Peakcircumferentialconv-leadingedge.csv')